In [2]:
!/usr/bin/env python3
# -*- coding: utf-8 -*-

from scipy.linalg.blas import zhemm
import numpy as np
import matplotlib.pyplot as plt
import warnings
import os
from scipy.interpolate import CubicSpline
from scipy.integrate import solve_ivp
from scipy.linalg import cho_factor, cho_solve, lu_factor, lu_solve
import time
import numpy as np

m_high = 1
m_low = -4
mdim = m_high - m_low + 1
ndim = 1
size = mdim * ndim
"""
... (주석) ...
- F_bar, K_bar, G 행렬과 q 프로파일을 바이너리 파일에서 읽어 보간.
- 최종 F, K 행렬은 히트맵 플롯 시점에 필요한 샘플에 대해서만 즉석에서 계산하여 성능 저하를 방지.
"""
import numpy as np
import struct
import os
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline

# ─────────────────────────────────────────────────────────────
# 1 & 2. 파일 읽기 관련 함수 (이전과 동일)
# ─────────────────────────────────────────────────────────────

def get_matrix_map(m, kind, n_blocks, mband_k=None, path=None):
    if kind in ('F', 'G'):
        for mb_test in range(m):
            count = sum(len(range(j, min(m, j + mb_test + 1))) for j in range(m))
            if count == n_blocks:
                mband = mb_test
                print(f"[{os.path.basename(path)}] mband = {mband} (으)로 추정 (F/G).")
                d = {k+1: (i,j) for k, (j,i) in enumerate(((j,i) for j in range(m) for i in range(j, min(m, j+mband+1))))}
                return d
        raise ValueError(f"F/G 행렬의 mband를 추정할 수 없습니다. (m={m}, n_blocks={n_blocks})")
    elif kind == 'K':
        if mband_k is None: raise ValueError("K 행렬을 처리하려면 mband_k 값을 지정해야 합니다.")
        print(f"[{os.path.basename(path)}] mband = {mband_k} (지정된 값) 사용 (K).")
        valid_element_count = sum(len(range(max(0, j-mband_k), min(m, j+mband_k+1))) for j in range(m))
        if n_blocks > valid_element_count:
            print(f"  > 경고: 파일에 기록된 블록 수({n_blocks})가 실제 유효한 원소 수({valid_element_count})보다 많습니다.")
        d = {}
        k = 0
        for j in range(m):
            for i in range(max(0, j - mband_k), min(m, j + mband_k + 1)):
                k += 1
                d[k] = (i, j)
        return d
    else: raise ValueError("kind는 'F', 'G', 'K' 중 하나여야 합니다.")

def read_bin(path, m, kind, mband_k=None):
    all_records = []
    with open(path, 'rb') as f:
        while True:
            header_bytes = f.read(4)
            if not header_bytes: break
            record_len = struct.unpack('<i', header_bytes)[0]
            if record_len > 0:
                all_records.append(np.frombuffer(f.read(record_len), dtype=np.float32))
            else:
                all_records.append(None)
            f.read(4)
    element_blocks = []
    current_block = []
    for rec in all_records:
        if rec is not None: current_block.append(rec)
        else:
            if current_block: element_blocks.append(np.array(current_block))
            current_block = []
    if not element_blocks: raise ValueError(f"파일에서 데이터 블록을 찾지 못했습니다: {path}")

    n_blocks, n_psi = len(element_blocks), element_blocks[0].shape[0]
    psi, q_profile = element_blocks[0][:, 0], element_blocks[0][:, 1]
    idx_map = get_matrix_map(m, kind, n_blocks, mband_k=mband_k, path=path)
    
    mats = np.zeros((n_psi, m, m), dtype=np.complex128)
    for k, (i, j) in idx_map.items():
        if k > n_blocks: continue
        mats[:, i, j] = element_blocks[k - 1][:, 2] + 1j * element_blocks[k - 1][:, 3]

    if kind in ('G'):
        il, jl = np.tril_indices(m, -1)
        mats[:, jl, il] = np.conj(mats[:, il, jl])
        
    return psi, q_profile, mats

# ────────────────────────────────────────
# 3. dedup, make_interp, plot_q_profile (이전과 동일)
# ────────────────────────────────────────
def dedup(psi, q, stacks):
    uniq_psi, idx = np.unique(psi, return_index=True)
    return uniq_psi, q[idx], {k: v[idx] for k, v in stacks.items()}

def make_interp(psi, mat_stack):
    cs_r = CubicSpline(psi, mat_stack.real, axis=0)
    cs_i = CubicSpline(psi, mat_stack.imag, axis=0)
    return lambda x, cr=cs_r, ci=cs_i: cr(x) + 1j * ci(x)

def plot_q_profile(psi, q, q_interpolant):
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(psi, q, 'o', label='Original Data Points', markersize=5)
    psi_fine = np.linspace(psi.min(), psi.max(), 300)
    ax.plot(psi_fine, q_interpolant(psi_fine), '-', label='Cubic Spline Interpolation')
    ax.set(title='Safety Factor (q) Profile', xlabel='Normalized Poloidal Flux (ψ)', ylabel='Safety Factor (q)')
    ax.legend(); ax.grid(True, linestyle='--', alpha=0.6)
    plt.show()

# ────────────────────────────────────────
# *** REVISED *** 4. 최종 행렬의 히트맵을 그리는 함수
# ────────────────────────────────────────
def plot_final_heatmaps(psi_coords, M, q_interp, F_bar_interp, G_interp, K_bar_interp, nsamp=4):
    """
    보간 함수들을 받아, 플롯에 필요한 psi 샘플에 대해서만 최종 행렬을 계산하고 히트맵을 그립니다.
    """
    names = ['F', 'G', 'K']
    samp = np.quantile(psi_coords, np.linspace(.05, .95, nsamp))
    
    fig, ax = plt.subplots(len(names), nsamp,
                        figsize=(4.5 * nsamp, 4 * len(names)),
                        constrained_layout=True, squeeze=False)
    
    I = np.identity(M.shape[0])

    for c, p in enumerate(samp):
        # --- 계산은 이 루프 안에서 즉석으로 수행됩니다 ---
        q_val = q_interp(p)
        Q_mat = M - q_val * I
        
        # 각 psi 샘플(p)에 대해 최종 행렬 계산
        final_mats = {
            'F': Q_mat @ F_bar_interp(p) @ Q_mat,
            'G': G_interp(p),
            'K': Q_mat @ K_bar_interp(p)
        }
        # ---------------------------------------------
        
        for r, n in enumerate(names):
            mat = final_mats[n]
            im = ax[r, c].imshow(np.abs(mat), cmap='viridis', origin='upper', aspect='equal')
            ax[r, c].set_title(f'{n}, ψ≈{p:.3f}')
            if r == len(names) - 1: ax[r, c].set_xlabel('j')
            if c == 0: ax[r, c].set_ylabel('i')
            plt.colorbar(im, ax=ax[r, c], label=f'|{n}|')
            
    fig.suptitle('Final F, G, K Heatmaps (Calculated On-the-fly)', y=1.03)
    plt.show()

# ────────────────────────────────────────
# 5. 메인 스크립트
# ────────────────────────────────────────
if __name__ == "__main__":
    # --- 사용자 설정 변수 ---
    MPERT = mdim
    MBAND_K = MPERT - 1 
    M_MODES = np.arange(m_low, m_high + 1) # MPERT=36일 경우, -17 ~ 18
    
    BASE_PATH = '/Users/seoda-eun/git/JPEC3/JPEC/src/DCON/validate'
    
    # --- 데이터 읽기 ---
    files_to_read = { 'F': 'fs.bin', 'G': 'gs.bin', 'K': 'ks.bin' }
    matrix_stacks, psi_coords, q_coords = {}, None, None
    for name, fname in files_to_read.items():
        path = os.path.join(BASE_PATH, fname)
        if not os.path.exists(path):
            print(f"파일을 찾을 수 없습니다: {path}"); continue
        print(f"파일 읽는 중: {fname} (as {'F_bar' if name=='F' else 'K_bar' if name=='K' else name})")
        try:
            mband_val = MBAND_K if name == 'K' else None
            psi, q, mat = read_bin(path, MPERT, name, mband_k=mband_val)
            matrix_stacks[name] = mat
            if psi_coords is None: psi_coords, q_coords = psi, q
        except Exception as e:
            print(f"오류: {fname} 처리 중 문제 발생. {e}"); import traceback; traceback.print_exc()

    if not matrix_stacks:
        print("데이터를 성공적으로 읽지 못했습니다. 프로그램을 종료합니다."); exit()

    # --- 데이터 처리 및 보간 ---
    if 'F' in matrix_stacks:
        print("\nF_bar 행렬 복원 (L·Lᴴ)...")
        L = np.tril(matrix_stacks['F'])                    # 하삼각(대각 포함)
        matrix_stacks['F'] = L @ L.transpose(0, 2, 1).conj()  # F_bar = L Lᴴ

    psi_coords, q_coords, matrix_stacks = dedup(psi_coords, q_coords, matrix_stacks)
    
    print("보간 함수 생성 중 (F_bar, G, K_bar, q)...")
    F_bar_interp = make_interp(psi_coords, matrix_stacks['F'])
    G_interp = make_interp(psi_coords, matrix_stacks['G'])
    K_bar_interp = make_interp(psi_coords, matrix_stacks['K'])
    q_of_psi = CubicSpline(psi_coords, q_coords)
    print("보간 함수 생성 완료.")
    
    # --- 최종 행렬 계산 및 시각화 ---
    if len(M_MODES) != MPERT:
        raise ValueError(f"M_MODES 배열 길이({len(M_MODES)})가 MPERT({MPERT})와 불일치.")
    M = np.diag(M_MODES)

    # 시각화: 플롯에 필요한 4개의 샘플에 대해서만 즉석에서 계산
    plot_q_profile(psi_coords, q_coords, q_of_psi)
    plot_final_heatmaps(psi_coords, M, q_of_psi, F_bar_interp, G_interp, K_bar_interp, nsamp=4)

    # --- 단일 psi 값에 대한 분석용 함수 (선택적 사용) ---
    def compute_matrices(psi):
        """특정 psi에 대한 최종 F, G, K 행렬을 계산합니다."""
        I = np.identity(MPERT)
        q_val = q_of_psi(psi)
        Q_mat = M - q_val * I
        
        f_final = Q_mat @ F_bar_interp(psi) @ Q_mat
        g_final = G_interp(psi)
        k_final = Q_mat @ K_bar_interp(psi)
        
        return {'F': f_final, 'G': g_final, 'K': k_final}

    print("\n분석이 완료되었습니다.")
    print("`compute_matrices(psi)` 함수로 특정 psi에서의 행렬을 계산할 수 있습니다.")
    psi_test = 0.5
    final_mats_at_half = compute_matrices(psi_test)
    print(f"예시: ψ = {psi_test} 에서의 |F_00|: {np.abs(final_mats_at_half['F'][0,0]):.4e}")

Python 3.12.8 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 11:37:13) [Clang 14.0.6 ] on darwin
Type "help", "copyright", "credits" or "license" for more information.
>>> ^C
>>> 
Traceback (most recent call last):
  File "<stdin>", line 0, in <module>
KeyboardInterrupt
>>> 

ModuleNotFoundError: No module named 'scipy'

In [1]:
conda install numpy

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=25.5.1



## Package Plan ##

  environment location: /Users/seoda-eun/anaconda3/envs/mkl_env

  added / updated specs:
    - numpy


The following packages will be UPDATED:

  ca-certificates    conda-forge::ca-certificates-2025.1.3~ --> pkgs/main::ca-certificates-2025.2.25-hecd8cb5_0 




Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Tuple, Dict
from scipy.interpolate import interp1d
import struct

class TokamakCylinderLoader:
    """
    A class to read and interpolate Tokamak and Cylinder matrices (A–H, F–K).

    Methods
    -------
    load_all(psin_target)
        Reads all Tokamak (A,B,C,D,E,H from imats; F from fs.bin; G from gs.bin; K from ks.bin)
        and Cylinder (A,B,C,D,E,H from individual CSVs; F,G,K from diag CSV), then interpolates
        onto psin_target.

    Attributes
    ----------
    imats_path       : str
        Path to tokamak 'imats.out'
    fs_bin_path      : str
        Path to 'fs.bin'
    gs_bin_path      : str
        Path to 'gs.bin'
    ks_bin_path      : str
        Path to 'ks.bin'
    mpert            : int
        mpert value used in Fortran
    mband            : int
        mband value used in Fortran
    cyl_paths        : Dict[str, str]
        Paths to cylinder A–H CSVs, keys 'A','B','C','D','E','H'
    cyl_FGK_diag_csv : str
        Path to 'dcon_FKG_diagonal_elements.csv'
    """
    
    def __init__(
        self,
        imats_path: str,
        fs_bin_path: str,
        gs_bin_path: str,
        ks_bin_path: str,
        mpert: int,
        mband: int,
        cyl_A_csv: str,
        cyl_B_csv: str,
        cyl_C_csv: str,
        cyl_D_csv: str,
        cyl_E_csv: str,
        cyl_H_csv: str,
        cyl_FGK_diag_csv: str
    ):
        self.imats_path = imats_path
        self.fs_bin_path = fs_bin_path
        self.gs_bin_path = gs_bin_path
        self.ks_bin_path = ks_bin_path
        self.mpert = mpert
        self.mband = mband
        self.cyl_paths = {
            'A': cyl_A_csv,
            'B': cyl_B_csv,
            'C': cyl_C_csv,
            'D': cyl_D_csv,
            'E': cyl_E_csv,
            'H': cyl_H_csv
        }
        self.cyl_FGK_diag_csv = cyl_FGK_diag_csv

    # 1. read_imats_out
    @staticmethod
    def read_imats_out(file_path: str, names: Tuple[str, ...] = ('A','B','C','D','E','H')) -> Tuple[np.ndarray, Dict[str, np.ndarray]]:
        with open(file_path, 'r') as f:
            lines = f.readlines()
        header = lines[2].split()
        mpsi  = int(header[2])
        mpert = int(header[-1])
        data  = lines[5:]
        m1 = [int(l.split()[1]) for l in data if len(l.split()) == 15]
        m2 = [int(l.split()[2]) for l in data if len(l.split()) == 15]
        mi, mj   = min(m1), max(m1)
        offset   = -mi
        size     = mj - mi + 1
        psi = np.zeros(mpsi + 1)
        mats = {n: np.zeros((mpsi + 1, size, size), np.complex128) for n in names}
        idx = {'A':0,'B':2,'C':4,'D':6,'E':8,'F':10,'G':12,'H':14}
        for ln in data:
            sp = ln.split()
            if len(sp) != 15:
                continue
            p = float(sp[0])
            i = int(sp[1]) + offset
            j = int(sp[2]) + offset
            k = int(round(p * mpsi))
            psi[k] = p
            nums = list(map(float, sp[3:]))
            for nm in names:
                r, imv = nums[idx[nm]], nums[idx[nm] + 1]
                mats[nm][k, i, j] = r + 1j * imv
        return psi, mats

    # 2. read_vector_out + expand_vector_to_diag
    @staticmethod
    def read_vector_out(file_path: str) -> Tuple[np.ndarray, np.ndarray]:
        blocks, cur = [], []
        for ln in Path(file_path).read_text().splitlines():
            s = ln.strip()
            if not s:
                if cur:
                    blocks.append(np.array(cur))
                    cur = []
                continue
            parts = s.split()
            if len(parts) == 4:
                p, _, re, im = parts
                cur.append((float(p), float(re) + 1j * float(im)))
        if cur:
            blocks.append(np.array(cur))
        psi = np.array([row[0] for row in blocks[0]])
        n_psi = len(psi)
        n_ipert = len(blocks)
        vec = np.empty((n_psi, n_ipert), np.complex64)
        for i, b in enumerate(blocks):
            vec[:, i] = b[:, 1]
        return psi, vec

    @staticmethod
    def expand_vector_to_diag(mat_vec: np.ndarray) -> np.ndarray:
        n_psi, n = mat_vec.shape
        full = np.zeros((n_psi, n, n), dtype=mat_vec.dtype)
        for k in range(n_psi):
            np.fill_diagonal(full[k], mat_vec[k])
        return full

    # 3. read_fs_bin / read_gs_bin / read_ks_bin
    @staticmethod
    def read_fs_bin(
        file_path: str,
        mpsi: int,
        mpert: int,
        mband: int
    ) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        n_ipert_fs = (mband + 1) * (2 * mpert - mband) // 2
        n_ipert_ks = (2 * mband + 1) * mpert
        bytes_per_ipert = (mpsi + 1) * (4 + 16 + 4) + (4 + 0 + 4)
        total_size = Path(file_path).stat().st_size
        if total_size % bytes_per_ipert == 0:
            cand_ipert = total_size // bytes_per_ipert
        else:
            raise ValueError(f"File size {total_size} not divisible by expected block size {bytes_per_ipert}.")
        if cand_ipert == n_ipert_fs:
            n_ipert = n_ipert_fs
        elif cand_ipert == n_ipert_ks:
            n_ipert = n_ipert_ks
        else:
            n_ipert = n_ipert_fs

        psi_arr = np.zeros(mpsi + 1, dtype=np.float64)
        sqfs   = np.zeros(mpsi + 1, dtype=np.float64)
        M      = np.zeros((n_ipert, mpsi + 1), dtype=np.complex128)

        with open(file_path, 'rb') as f:
            for ipert in range(n_ipert):
                for ipsi in range(mpsi + 1):
                    rec_len_bytes = f.read(4)
                    if len(rec_len_bytes) < 4:
                        raise EOFError("Unexpected EOF reading record length.")
                    rec_len = struct.unpack('<i', rec_len_bytes)[0]
                    blob = f.read(4 * 4)
                    vals = struct.unpack('<4f', blob)
                    _ = f.read(4)

                    if ipert == 0:
                        psi_arr[ipsi] = float(vals[0])
                        sqfs[ipsi]   = float(vals[1])
                    M[ipert, ipsi] = float(vals[2]) + 1j * float(vals[3])

                zero1 = f.read(4)
                if not zero1:
                    raise EOFError("Unexpected EOF reading blank record marker.")
                _ = f.read(4)

        return psi_arr, sqfs, M

    read_gs_bin = read_fs_bin
    read_ks_bin = read_fs_bin

    # 4. read_block_csv_matrix + read_cylinder_real_matrix
    @staticmethod
    def read_block_csv_matrix(file_path: str, want_imag: bool = False) -> Tuple[np.ndarray, np.ndarray]:
        psi_vals, blocks, cur, reading = [], [], [], None
        for ln in Path(file_path).read_text().splitlines():
            s = ln.strip()
            if not s:
                if cur:
                    blocks.append(np.array(cur, float))
                    cur = []
                continue
            if s.startswith('# psi='):
                psi_vals.append(float(s.split('=')[1]))
            elif s.startswith('# Real'):
                reading = 'real'
            elif s.startswith('# Imag'):
                reading = 'imag'
            elif s and not s.startswith('#'):
                if (reading == 'imag') == want_imag:
                    cur.append(list(map(float, s.split(','))))
        if cur:
            blocks.append(np.array(cur, float))
        psi = np.array(psi_vals)
        mats = np.stack(blocks, axis=0)
        return psi, mats

    @staticmethod
    def read_cylinder_real_matrix(file_path: str) -> Tuple[np.ndarray, np.ndarray]:
        psi, real_mat = TokamakCylinderLoader.read_block_csv_matrix(file_path, want_imag=False)
        complex_mat = real_mat.astype(np.complex128)
        return psi, complex_mat

    # 5. read_cylinder_FGK_from_diag
    @staticmethod
    def read_cylinder_FGK_from_diag(file_path: str) -> Tuple[np.ndarray, Dict[str, np.ndarray]]:
        df  = pd.read_csv(file_path)
        psi = df['psi'].values
        mats = {}
        for key in ('F','G','K'):
            prefix = f"{key}_diag"
            cols = [col for col in df.columns if col.startswith(prefix)]
            diag_vals = df[cols].values
            n_psi, m = diag_vals.shape
            full = np.zeros((n_psi, m, m), dtype=np.complex128)
            for k in range(n_psi):
                np.fill_diagonal(full[k], diag_vals[k])
            mats[key] = full
        return psi, mats

    # 6. clean_and_deduplicate_psin
    @staticmethod
    def clean_and_deduplicate_psin(psin: np.ndarray, mat: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        psin_rounded = np.round(psin, 8)
        unique_vals, inv_idx = np.unique(psin_rounded, return_inverse=True)
        m = mat.shape[1]
        mat_out = np.zeros((len(unique_vals), m, m), dtype=mat.dtype)
        counts = np.zeros(len(unique_vals), dtype=int)
        for i, idx in enumerate(inv_idx):
            mat_out[idx] += mat[i]
            counts[idx] += 1
        mat_out /= counts[:, None, None]
        return unique_vals, mat_out

    # 7. interpolate_matrix_over_psin
    @staticmethod
    def interpolate_matrix_over_psin(psin_orig: np.ndarray, mat_orig: np.ndarray, psin_target: np.ndarray) -> np.ndarray:
        n, m, _ = mat_orig.shape
        mat_interp = np.zeros((len(psin_target), m, m), dtype=np.complex128)
        for i in range(m):
            for j in range(m):
                f = interp1d(psin_orig, mat_orig[:, i, j], kind='cubic', bounds_error=False, fill_value="extrapolate")
                mat_interp[:, i, j] = f(psin_target)
        return mat_interp

    # 8. load_all: 통합 함수
    def load_all(self, psin_target: np.ndarray) -> Dict[str, Dict[str, np.ndarray]]:
        result = {'tokamak': {}, 'cylinder': {}}

        # --- Tokamak: A–H from imats.out ---
        psi_imats, mats_imats = TokamakCylinderLoader.read_imats_out(self.imats_path, names=('A','B','C','D','E','H'))
        for nm in ('A','B','C','D','E','H'):
            psi_clean, m_clean = TokamakCylinderLoader.clean_and_deduplicate_psin(psi_imats, mats_imats[nm])
            result['tokamak'][nm] = TokamakCylinderLoader.interpolate_matrix_over_psin(psi_clean, m_clean, psin_target)

        # --- Tokamak: F from fs.bin ---
        psi_fs, sqfs, mat_F = TokamakCylinderLoader.read_fs_bin(self.fs_bin_path, mpsi=psi_imats.size-1, mpert=self.mpert, mband=self.mband)
        psi_fs, mat_F = TokamakCylinderLoader.clean_and_deduplicate_psin(psi_fs, mat_F)
        result['tokamak']['F'] = TokamakCylinderLoader.interpolate_matrix_over_psin(psi_fs, mat_F, psin_target)

        # --- Tokamak: G from gs.bin ---
        psi_gs, sqgs, mat_G = TokamakCylinderLoader.read_gs_bin(self.gs_bin_path, mpsi=psi_imats.size-1, mpert=self.mpert, mband=self.mband)
        psi_gs, mat_G = TokamakCylinderLoader.clean_and_deduplicate_psin(psi_gs, mat_G)
        result['tokamak']['G'] = TokamakCylinderLoader.interpolate_matrix_over_psin(psi_gs, mat_G, psin_target)

        # --- Tokamak: K from ks.bin ---
        psi_ks, sqks, mat_K = TokamakCylinderLoader.read_ks_bin(self.ks_bin_path, mpsi=psi_imats.size-1, mpert=self.mpert, mband=self.mband)
        psi_ks, mat_K = TokamakCylinderLoader.clean_and_deduplicate_psin(psi_ks, mat_K)
        result['tokamak']['K'] = TokamakCylinderLoader.interpolate_matrix_over_psin(psi_ks, mat_K, psin_target)

        # --- Cylinder: A–H from CSVs ---
        for nm, path in self.cyl_paths.items():
            psi_c, mat_c = TokamakCylinderLoader.read_cylinder_real_matrix(path)
            psi_c, mat_c = TokamakCylinderLoader.clean_and_deduplicate_psin(psi_c, mat_c)
            result['cylinder'][nm] = TokamakCylinderLoader.interpolate_matrix_over_psin(psi_c, mat_c, psin_target)

        # --- Cylinder: F, G, K from diag CSV ---
        psi_fkg, mats_fkg = TokamakCylinderLoader.read_cylinder_FGK_from_diag(self.cyl_FGK_diag_csv)
        for nm in ('F','G','K'):
            psi_c, mat_c = TokamakCylinderLoader.clean_and_deduplicate_psin(psi_fkg, mats_fkg[nm])
            result['cylinder'][nm] = TokamakCylinderLoader.interpolate_matrix_over_psin(psi_c, mat_c, psin_target)

        return result

# ======================
# Usage example
# ======================
if __name__ == "__main__":
    # 1) 환경에 맞게 경로와 Fortran 파라미터 설정
    loader = TokamakCylinderLoader(
        imats_path       = "imats.out",
        fs_bin_path      = "fs.bin",
        gs_bin_path      = "gs.bin",
        ks_bin_path      = "ks.bin",
        mpert            = 6,
        mband            = 2,
        cyl_A_csv        = "A_matrices.csv",
        cyl_B_csv        = "B_matrices.csv",
        cyl_C_csv        = "C_matrices.csv",
        cyl_D_csv        = "D_matrices.csv",
        cyl_E_csv        = "E_matrices.csv",
        cyl_H_csv        = "H_matrices.csv",
        cyl_FGK_diag_csv = "dcon_FKG_diagonal_elements.csv"
    )

    # 2) 공통 psin 그리드 정의
    psin_target = np.linspace(0, 1, 200)

    # 3) 모든 행렬 읽고 보간
    all_data = loader.load_all(psin_target)

    # 4) 결과 출력
    print("=== Tokamak matrices ===")
    for name, mat in all_data['tokamak'].items():
        print(f"{name}: {mat.shape}")
    print("\n=== Cylinder matrices ===")
    for name, mat in all_data['cylinder'].items():
        print(f"{name}: {mat.shape}")


: 